In [1]:
from decimal import Decimal

from btopt.data.dataloader import CSVDataLoader
from btopt.data.timeframe import Timeframe
from btopt.engine import Engine
from btopt.order import Order
from btopt.strategy.strategy import Strategy

In [2]:
class SimpleMovingCrossover(Strategy):
    def __init__(self, name, parameters):
        super().__init__(name, parameters)
        self.short_period = self.parameters.get("short_period", 10)
        self.long_period = self.parameters.get("long_period", 20)

    def generate_signals(self, bar):
        symbol = bar.ticker
        data = self.get_data(symbol, self.primary_timeframe)

        if len(data.close) < self.long_period:
            return []

        short_ma = data.close[-self.short_period :].mean()
        long_ma = data.close[-self.long_period :].mean()

        current_position = self.get_current_position(symbol)
        signals = []

        if short_ma > long_ma and current_position <= 0:
            signals.append(
                {
                    "symbol": symbol,
                    "action": "BUY",
                    "size": 1,
                    "price": bar.close,
                    "order_type": Order.ExecType.MARKET,
                }
            )
        elif short_ma < long_ma and current_position >= 0:
            signals.append(
                {
                    "symbol": symbol,
                    "action": "SELL",
                    "size": 1,
                    "price": bar.close,
                    "order_type": Order.ExecType.MARKET,
                }
            )

        return signals

    def on_order(self, order):
        print(f"Order update: {order}")

    def on_trade(self, trade):
        print(f"Trade update: {trade}")

In [3]:
def run_backtest():
    # Initialize the engine
    engine = Engine()

    # Load data
    symbol = "EURUSD"
    start_date = "2022-01-01"
    end_date = "2023-01-01"
    timeframe = "1d"

    dataloader = CSVDataLoader(symbol, "1m", start_date=start_date, end_date=end_date)
    engine.resample_data(dataloader, timeframe)

    # Create and add the strategy
    strategy_params = {"short_period": 10, "long_period": 20}
    strategy = SimpleMovingCrossover("SMA Crossover", strategy_params)
    engine.add_strategy(strategy, primary_timeframe=Timeframe("1d"))

    # Set up the backtest configuration
    initial_capital = Decimal("100000")
    commission_rate = Decimal("0.001")  # 0.1% commission
    config = {
        "initial_capital": initial_capital,
        "commission_rate": commission_rate,
    }
    engine.set_config(config)

    # Run the backtest
    results = engine.run()

    # Print the results
    print("\nBacktest Results:")
    print(f"Total Return: {results['performance_metrics']['total_return']:.2f}%")
    print(f"Sharpe Ratio: {results['performance_metrics']['sharpe_ratio']:.2f}")
    print(f"Max Drawdown: {results['performance_metrics']['max_drawdown']:.2f}%")
    print(f"Win Rate: {results['performance_metrics']['win_rate']:.2f}%")
    print(f"Profit Factor: {results['performance_metrics']['profit_factor']:.2f}")
    print(f"Total Trades: {results['performance_metrics']['total_trades']}")

    # Plot the equity curve
    equity_curve = results["equity_curve"]
    equity_curve.set_index("timestamp", inplace=True)
    equity_curve["equity"].plot(title="Equity Curve")
    import matplotlib.pyplot as plt

    plt.show()

In [4]:
run_backtest()

Data updated for EURUSD at 1d timeframe.
Data resampled and added for EURUSD from 1m to 1d timeframe.
Resampled data for 1 symbols to 1d.


AttributeError: 'Engine' object has no attribute '_strategy_timeframes'